In [2]:
import random
import os
import requests
import pandas as pd
import json
import time
from PIL import Image 
from IPython.display import display 
from pathlib import Path
from dotenv import load_dotenv

In [3]:
# Adding images to Pinata and retrieving the hash

    #Loading the ENV
load_dotenv()

PINATA_API_KEY = os.getenv('PINATA_API_KEY')
PINATA_SECRET_API_KEY = os.getenv('PINATA_SECRET_API_KEY')

PINATA_BASE_URL = 'https://api.pinata.cloud/'
endpoint = 'pinning/pinFileToIPFS'
# Change this to upload a different file
filepath = './images/'
headers = {'pinata_api_key': PINATA_API_KEY,
           'pinata_secret_api_key': PINATA_SECRET_API_KEY}

image_hashes = []

for subdir, dirs, files in os.walk(filepath):
    for file in files:
        with Path((os.path.join(filepath + file))).open("rb") as fp:
            image_binary = fp.read()
            response = requests.post(PINATA_BASE_URL + endpoint,
                                     files={"file": ((os.path.join(file)), image_binary)},
                                     headers=headers)
            pinata_response = json.loads(response.text)
            image_hashes.append(pinata_response["IpfsHash"])

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 2


KeyError: 'IpfsHash'

In [ ]:
# Adding metadata to Pinata and creating a CSV with the hashes

pinata_gateway = "https://gateway.pinata.cloud/ipfs/"
pinata_hashes = [pinata_gateway + x for x in image_hashes]

filepath2 = './metadata/'

metadata_hashes = []
metadata_gateway = []
file_list = []

for subdir, dirs, files in os.walk(filepath2):
    for file in files:
        file_list.append(file)
        with open(filepath2 + file, 'r') as f:
            data = (f.readlines())
            flist_length = (len(file_list) - 1)
            data[1] = '    "image": ' + '"' + str(pinata_hashes[flist_length]) + '"' + ',\n'
           
        with open(filepath2 + file, 'w') as f:
            f.writelines(data)
        
        with Path((os.path.join(filepath2 + file))).open("rb") as fp:
            image_binary = fp.read()
            response = requests.post(PINATA_BASE_URL + endpoint,
                                     files={"file": ((os.path.join(file)), image_binary)},
                                     headers=headers)
            pinata_response2 = json.loads(response.text)
            file_number2 = file.rsplit( ".", 1 )[0]
            metadata_hashes.append(pinata_response2["IpfsHash"])
            metadata_gateway = [pinata_gateway + x for x in metadata_hashes]
            mg_csv = pd.DataFrame(metadata_gateway)
            mg_csv.to_csv('metadata_gateway.csv', index=False)

In [ ]:
# Deleting from Pinata

endpoint2 = 'pinning/unpin/'
hashToDelete = metadata_hashes # replace with needed hash list
payload = {}

for i in range(len(hashToDelete)):
   response = requests.request("DELETE", PINATA_BASE_URL + endpoint2 +
       str(hashToDelete[i]), headers=headers, data=payload)
   print(response.text)
   time.sleep(0.5) # Used to prevent API timeout

In [ ]:
# Each image is made up a series of traits
# The weightings for each trait drive the rarity and add up to 100%

# Background weights for hidden props

background = ["Blue", "Red", "Purple", "Orange"] 
background_weights = [70, 20, 8, 2]

body_type = ["Male", "Female", "Tiger", "Mutant", "Alien", "Tulip"]
body_type_weights = [60, 38.625, 1, 0.25, 0.1, 0.025]

over_eye = ["Eye", "Eye_Cover"]
over_eye_weights = [75, 25]

over_mouth = ["Yes_Mouth", "Over_Mouth"]
over_mouth_weights = [80, 20]

hat_type = ["Hat", "Hair"]
hat_type_weights = [60, 40]

Cy_Faces = ["Face_Blue", "Face_Green", "Face_Pink", "Face_Red"]

hat_combo_f = ["Amulet", "Bow", "Asian", "Japanese", 
               "Crown", "Blue_Cap", "McCap", "Red_Cap"]

hair_combo_f = ["Brown_Short", "Black_Long", 
                "Blonde_Long", "Ponytail","Purple_Short", 
                "Red_Hair", "Black_Short", "Blonde_Short",
                "Blonde_Short2", "Purple",]

hat_combo_m = ["Asian", "Japanese", "Crown", 
               "Blue_Cap", "McCap", "Red_Cap", "Hood"]

hair_combo_m = ["Braided", "Emo", "Parted"]

###########################################################################

# Image layers and weights

body = ["Cream", "Brown", "Silver", "Black", "Red",
        "White", "Green", "Tiger", "Dirt"]
body_weights = [35, 28, 21, 12, 2, 0, 0, 0, 0]

eye = ["Null_Eye", "Black", "Purple", "Cute", 
       "Blue", "Red", "Alien", "Red_Laser", "Blue_Laser"] 
eye_weights = [0, 60, 25, 8, 4, 3, 0, 0, 0]

laser_eye = ["Red_Laser", "Blue_Laser"]
laser_eye_weights = [55, 45]

eye_c = ["Null_EyeC", "3D_Glasses", "Orange_Shades", "Purple_Shades", 
         "EZ_Shades", "Red_Blindfold", "White_Blindfold", "VR_Goggles", 
         "TV_Goggles", "Aviators_Black", "Aviators_Gold",
         "Doc_Shades", "Cy_Orange", "Cy_Purple", "Night_Vision", "Cy_PurpleT"]
eye_c_weights = [0, 10, 13, 11, 2, 2, 5, 10, 5, 15, 10, 5, 6, 4, 2, 0]

eye_a = ["Null_EyeA", "Bandit", "Eye_Patch", "Gold_Monocle", "Silver_Monocle",
         "Nerd_Glasses", "Pink_Glasses", "Orange_Glasses", "Purple_Glasses",
         "Goggles",  "Scouter", "Laser_EyePatch", "Laser_EyePatchB",
         "Cy_Blue", "Cy_Red", "Cy_Green"]
eye_a_weights = [36, 10, 11, 7, 6, 1, 1, 2, 4, 6, 5, 0, 0, 4, 3, 3]

mouth = ["Null_Mouth", "Neutral", "Smile", "Munch"]
mouth_weights = [0, 93, 5, 2]

mouth_a = ["Null_MouthA", "Cigarette", "E_Cig",
           "Lolipop", "Pipe", "Tongue", "Cigar", "Gum"]
mouth_a_weights = [60, 11, 7, 10, 2, 5, 3, 2]

face_cover = ["Null_FaceC", "Blue_Bandana", "Orange_Bandana", "Mask",
              "Face_Blue", "Face_Green", "Face_Pink", "Face_Red",
              "Gas_Mask", "Face_BlueT", "Face_RedT"]
face_cover_weights = [0, 20, 17, 10, 8, 7, 5, 5, 5, 0, 0]

facial_h = ["Null_FacialH", "Big_Beard", "Goatee", "Mustache", "Mustache2", 
            "Small_Beard", "5oclock", "Soulpatch", "Stash", "Patch",
            "Scar"]
facial_h_weights = [35, 3, 11, 8, 10, 10, 5, 9, 5, 2, 2]

nose = ["Null_Nose", "Black_Nose", "Red_Nose", "Tiger_Nose", "Alien_Nose", "Tulip_Nose"]
nose_weights = [0, 99, 1, 0, 0, 0]

hat = ["Null_Hat", "Beanie", "Big_Hat", "Blue_Cap", "McCap", "Blue_Hat",
       "Gray_Hat", "Blue_Band", "Red_Band", "Headband", "Bow", "Bowler", 
       "Cowboy", "Crown", "Blue_Dorag", "Orange_Dorag", "Goggles",
       "Prop", "Asian", "Sheriff", "Robinhood", "Goggles_Hat",
       "Top_Hat", "Earing", "Hood", "Amulet", "Red_Cap", "Japanese",
       "Blue_Tulip", "Orange_Tulip", "Pink_Tulip"]
hat_weights = [5, 5, 5, 5, 2 ,5, 5, 2, 2, 4, 2, 
               6, 4, 1, 4, 4, 3, 4, 6, 
               6, 7, 4, 1, 2, 1, 1, 2, 2, 0, 0, 0]

hair_m = ["Null_HairM", "Blackhawk", "Bluehawk", "Redhawk", 
          "Fire", "Smoke", "Mohawk_G", "Mohawk_P",
          "SSB", "SSR", "SSY", "Braided", "Emo", "Bluehair",
          "Brownhawk", "Messyhair", "Orangehair", "Greenhair", "Vlad",
          "Parted"]
hair_m_weights = [0, 12, 3, 5, 3, 4, 12, 5, 1,
                  2, 3, 7, 12, 3, 4, 5, 2, 2, 5, 10]

hair_f = ["Null_HairF", "Brown_Short", "Black_Long", "Blonde_Long", "Ponytail",
          "Purple_Short", "Red_Hair", "Black_Short", "Blonde_Short",
          "Blonde_Short2", "Purple", "White_Crazy", "Black_Hair"]
hair_f_weights = [0, 13, 15, 10, 9, 7, 5, 17, 5, 8, 1, 5, 5]

tulip = ["Blue_Tulip", "Orange_Tulip", "Pink_Tulip"]
tulip_weights = [33, 33, 34]

# Dictionary variable for each trait. 
# Each trait corresponds to its file name

background_files = {
    "Blue": "blue",
    "Red": "red",
    "Purple": "purple",
    "Orange": "orange",
}

body_files = {
    "Cream" : "base",
    "Brown" : "brown", 
    "Silver" : "gray", 
    "Black" : "black", 
    "Red" : "red",
    "Alien_Body" : "white", 
    "Mutant_Body" : "green",
    "Tiger_Body" : "tiger",
    "Dirt_Body" : "dirt",
}

eye_files = {
    "Null_Eye" : "Null",
    "Black" : "eye_base",
    "Blue" : "eye_bluel",
    "Red" : "eye_redl",
    "Purple" : "eye_purple",
    "Cute" : "eye_cute", 
    "Alien" : "eye_sparkle",
    "Red_Laser" : "eye_laser",
    "Blue_Laser" : "eye_laserb",
}

eye_c_files = {
    "Null_EyeC" : "Null",
    "3D_Glasses" : "3dglasses", 
    "Orange_Shades" : "whiteglasses", 
    "Purple_Shades" : "blackglasses",
    "EZ_Shades" : "ezglasses",
    "Red_Blindfold" : "redbf", 
    "White_Blindfold" : "whitebf",
    "VR_Goggles" : "goggles",
    "TV_Goggles" : "gogglestv",
    "Aviators_Black" : "blacka", 
    "Aviators_Gold" : "golda",
    "Doc_Shades" : "docglass",
    "Cy_Orange" : "cyorange", 
    "Cy_Purple" : "cypurple",
    "Night_Vision" : "nightvision",
    "Cy_PurpleT" : "cypurplet",
}

eye_a_files = {
    "Null_EyeA" : "Null", 
    "Bandit" : "bandit",
    "Eye_Patch" : "eyepatch",
    "Gold_Monocle" : "monocleg",
    "Silver_Monocle" : "monocles",
    "Nerd_Glasses" : "nerdglass",
    "Pink_Glasses" : "pinkglass",
    "Goggles" : "spgoggles",
    "Scouter" : "scouter",
    "Laser_EyePatch" : "laserpatch",
    "Laser_EyePatchB" : "laserpatchb",
    "Orange_Glasses" : "clearglass",
    "Purple_Glasses" : "purpleglass",
    "Cy_Blue" : "cyblue", 
    "Cy_Red" : "cyred",
    "Cy_Green" : "cygreen",
}

mouth_files = {
    "Null_Mouth" : "Null",
    "Neutral" : "base",
    "Smile" : "smile",
    "Munch" : "munch",
}

mouth_a_files = {
    "Null_MouthA" : "Null",
    "Cigarette" : "cig",
    "E_Cig" : "ecig",
    "Lolipop" : "lolipop",
    "Pipe" : "pipe",
    "Tongue" : "tongue",
    "Cigar" : "cigar",
    "Gum" : "gum",
}

face_cover_files = {
    "Null_FaceC" : "Null",
    "Blue_Bandana" : "bandanab", 
    "Orange_Bandana" : "bandanao", 
    "Mask" : "mask",
    "Face_Blue" : "blueface", 
    "Face_Green" : "greenface",
    "Face_Pink" : "pinkface",
    "Face_Red" : "redface",
    "Gas_Mask" : "gasmask",
    "Face_BlueT" : "bluefacet",
    "Face_RedT" : "redfacet",
}

facial_h_files = {
    "Null_FacialH" : "Null",
    "Big_Beard" : "beardbig", 
    "Goatee" : "goatee",
    "Mustache" : "mustache",
    "Mustache2" : "mustache2", 
    "Small_Beard" : "smallbeard",
    "5oclock" : "5oclock",
    "Soulpatch" : "soulpatch", 
    "Stash" : "stash",
    "Patch" : "lightning",
    "Scar" : "scar",
}

nose_files = {
    "Null_Nose" : "Null",
    "Black_Nose" : "nose_base",
    "Red_Nose" : "nose_red",
    "Tiger_Nose" : "nose_tiger",
    "Alien_Nose" : "nose_alien",
    "Tulip_Nose" : "nose_tulip",
}

hat_files = {
    "Null_Hat" : "Null",
    "Beanie" : "beanie",
    "Big_Hat" : "bighat",
    "Blue_Cap" : "bluecap",
    "McCap" : "mccap",
    "Blue_Hat" : "bluehat",
    "Gray_Hat" : "grayhat",
    "Blue_Band" : "bluehband",
    "Red_Band" : "redhband", 
    "Headband" : "hband",
    "Bow" : "bow",
    "Bowler" : "bowlerhat", 
    "Cowboy" : "cowboyhat",
    "Crown" : "crown",
    "Blue_Dorag" : "dooragb",
    "Orange_Dorag" : "doorago",
    "Goggles" : "gogglesp",
    "Prop" : "prophat",
    "Asian" : "ricehat",
    "Sheriff" : "rickhat",
    "Robinhood" : "robinhat",
    "Goggles_Hat" : "sphat",
    "Top_Hat" : "tophat",
    "Earing" : "earing", 
    "Hood" : "hood",
    "Amulet" : "amulet",
    "Red_Cap" : "redcap",
    "Japanese" : "japhat",
    "Blue_Tulip" : "bluetulip", 
    "Orange_Tulip" : "orangetulip",
    "Pink_Tulip" : "pinktulip",
}

hair_m_files = {
    "Null_HairM" : "Null",
    "Blackfro" : "blackfro", 
    "Whitefro" : "whitefro", 
    "Blackhawk" : "blackhawk", 
    "Bluehawk" : "bluehawk",
    "Redhawk" : "redhawk", 
    "Fire" : "firehair",
    "Smoke" : "smokehair",
    "Mohawk_G" : "mohawkg",
    "Mohawk_P" : "mohawkp",
    "SSB" : "ssb", 
    "SSR" : "ssr",
    "SSY" : "ssy",
    "Braided" : "travisscott",
    "Emo" : "sadge",
    "Bluehair" : "bluehair",
    "Brownhawk" : "brownhawk", 
    "Messyhair" : "messyhair",
    "Orangehair" : "orangehair",
    "Greenhair" : "rick",
    "Vlad" : "vlad",
    "Parted" : "tiktok",
}

hair_f_files = {
    "Null_HairF" : "Null",
    "Brown_Short" : "brownshort", 
    "Black_Long" : "longblack", 
    "Blonde_Long" : "longblonde", 
    "Ponytail" : "ponytail",
    "Purple_Short" : "purpleshort",
    "Red_Hair" : "redhair",
    "Black_Short" : "shortblack",
    "Blonde_Short" : "shortblonde",
    "Blonde_Short2" : "shortblonde2", 
    "Purple" : "sombra",
    "White_Crazy" : "whitehair",
    "Black_Hair" : "blackhair",
}

In [ ]:
## Generate Traits

TOTAL_IMAGES = 10000 # Number of random unique images we want to generate

all_images = [] 

# A recursive function to generate unique image combinations
def create_new_image():
    
    new_image = {} # Dictionary for compilation of new image traits

    # Creating trait in for dictionary and picking unique choices for each 
    new_image ["Background"] = random.choices(background, background_weights)[0]
    new_image ["Body"] = random.choices(body, body_weights)[0]
    new_image ["BodyType"] = random.choices(body_type, body_type_weights)[0]
    new_image ["Eye"] = random.choices(eye, eye_weights)[0]
    new_image ["EyeCover"] = random.choices(eye_c, eye_c_weights)[0]
    new_image ["EyeA"] = random.choices(eye_a, eye_a_weights)[0]
    new_image ["FacialH"] = random.choices(facial_h, facial_h_weights)[0]
    new_image ["Mouth"] = random.choices(mouth, mouth_weights)[0]
    new_image ["MouthA"] = random.choices(mouth_a, mouth_a_weights)[0]
    new_image ["Nose"] = random.choices(nose, nose_weights)[0]
    new_image ["FaceCover"] = random.choices(face_cover, face_cover_weights)[0]
    new_image ["HairM"] = random.choices(hair_m, hair_m_weights)[0]
    new_image ["HairF"] = random.choices(hair_f, hair_f_weights)[0]
    new_image ["Hat"] = random.choices(hat, hat_weights)[0]
    
    # Picks if the eye will be covered or not
    eye_choice = random.choices(over_eye, over_eye_weights)[0]
    if eye_choice == "Eye_Cover":
        new_image ["Eye"] = "Null_Eye"
        new_image ["EyeA"] = "Null_EyeA"
    elif eye_choice == "Eye":
        new_image ["EyeCover"] = "Null_EyeC"
    
    # Fixes Munch + Stash conflicts
    if new_image ["FacialH"] == "Stash" and new_image ["Mouth"] == "Munch":
        new_image ["FacialH"] = "Null_FacialH"
    
    # Removes Facial Hair if tongue is present
    if new_image ["MouthA"] == "Tongue":
        new_image ["FacialH"] = "Null_FacialH"
    
    # Removes Bandit & EyeCovers if Patch is present
    if (new_image ["EyeA"] == "Bandit" or 
        new_image ["EyeCover"] != "Null_EyeC") and new_image ["MouthA"] == "Patch":
        new_image ["EyeA"] = "Null_EyeA"
        
    if new_image ["EyeA"] == "Bandit" and new_image ["Eye"] == "Cute":
        new_image ["EyeA"] = "Null_EyeA"
        
    # Picks if mouth and nose are covered
    over_mouth_choice = random.choices(over_mouth, over_mouth_weights)[0]
    if over_mouth_choice == "Over_Mouth":
        new_image["Mouth"] = "Null_Mouth"
        new_image["Nose"] = "Null_Nose"
        new_image["FacialH"] = "Null_FacialH"
        new_image ["MouthA"] = "Null_MouthA"     
    elif over_mouth_choice == "Yes_Mouth":
        new_image["FaceCover"] = "Null_FaceC"
        
    def CyFace():
        new_image ["Mouth"] = "Null_Mouth"
        new_image ["Nose"] = "Null_Nose"
        new_image ["FacialH"] = "Null_FacialH"
        new_image ["MouthA"] = "Null_MouthA"
        new_image ["Eye"] = "Null_Eye"
        new_image ["EyeA"] = "Null_EyeA"
        new_image ["EyeCover"] = "Null_EyeC"
    
    # Changes conflicting items to Null if Cyborg Face Cover
    if new_image ["FaceCover"] == "Face_Blue":
        CyFace()
    elif new_image ["FaceCover"] == "Face_Green":
        CyFace()
    elif new_image ["FaceCover"] == "Face_Red":
        CyFace()
    elif new_image ["FaceCover"] == "Face_Pink":
        CyFace()
    
    # Fixes Goggle face + Goggle hat conflicts
    if new_image ["Hat"] == "Goggles_SP" and new_image ["EyeA"] == "Goggles_SP":
        new_image ["EyeA"] = "Null_EyeA"
        
    # Chooses hat types + combo
    hat_type_choice = random.choices(hat_type, hat_type_weights)[0]
    
    # Chooses Body Type
    body_type_choice = new_image ["BodyType"]
    
    # Male Conflicts
    if body_type_choice == "Male":
        new_image ["HairF"] = "Null_HairF"
        if hat_type_choice == "Hair":
            new_image ["HairF"] = "Null_HairF"
            new_image ["Hat"] = "Null_Hat"
        elif hat_type_choice == "Hat" and (new_image ["HairM"] in hair_combo_m) and (new_image ["Hat"] in hat_combo_m):
            new_image ["HairF"] = "Null_HairF"
        else:
            new_image ["HairF"] = "Null_HairF"
            new_image ["HairM"] = "Null_HairM"
    
    # Female conflicts
    elif body_type_choice == "Female": 
        new_image ["HairM"] = "Null_HairM"
        new_image ["FacialH"] = "Null_FacialH"
        if hat_type_choice == "Hair":
            new_image ["HairM"] = "Null_HairM"
            new_image ["Hat"] = "Null_Hat"
        elif hat_type_choice == "Hat" and (new_image ["HairF"] in hair_combo_f) and (new_image ["Hat"] in hat_combo_f):
            new_image ["HairM"] = "Null_HairM"
        else:
            new_image ["HairF"] = "Null_HairF"
            new_image ["HairM"] = "Null_HairM"
        if new_image ["Body"] == "Red" and new_image ["HairF"] == "Red_Hair":
            new_image ["HairF"] = "Black_Hair"
            
        
    # Tiger conflicts
    elif body_type_choice == "Tiger":
        new_image ["Body"] = "Tiger"
        new_image ["Nose"] = "Tiger"
        new_image ["HairM"] = "Null_HairM"
        new_image ["HairF"] = "Null_HairF"
        if new_image ["FaceCover"] != "Null_FaceC":
            new_image ["Nose"] = "Null_Nose"
        if new_image ["FaceCover"] == "Face_Blue":
            new_image ["FaceCover"] = "Face_BlueT"
        if new_image ["FaceCover"] == "Face_Red":
            new_image ["FaceCover"] = "Face_RedT"
        if new_image ["EyeCover"] == "Cy_Purple":
            new_image ["EyeCover"] = "Cy_PurpleT"
    
    # Radioactive Conflicts
    elif body_type_choice == "Mutant":
        new_image ["FacialH"] = "Null_FacialH"
        new_image ["HairM"] = "Null_HairM"
        new_image ["HairF"] = "Null_HairF"
        new_image ["Body"] = "Green"
        new_image ["Eye"] = random.choices(laser_eye, laser_eye_weights)[0]
        if new_image ["FaceCover"] in Cy_Faces:
            new_image ["Mouth"] = "Neutral"
            new_image ["Nose"] = random.choices(nose, nose_weights)[0]
            new_image ["Eye"] = random.choices(laser_eye, laser_eye_weights)[0]
            new_image ["FaceCover"] = "Null_FaceC"
        if new_image ["EyeCover"] != "Null_EyeC":
            new_image ["Eye"] = "Null_Eye"
            
    # If Laser Eye + Eye Patch are present then new input will be set
    if new_image ["Eye"] == "Laser" and new_image ["EyeA"] == "Eye_Patch":
        new_image ["Eye"] = "Null_Eye"
        new_image ["EyeA"] = "Laser_Patch"
    elif new_image ["Eye"] == "Laser_B" and new_image ["EyeA"] == "Eye_Patch":
        new_image ["Eye"] = "Null_Eye"
        new_image ["EyeA"] = "Laser_PatchB"
    
    # If Laser Eye + Bandit Accessory are present then Bandit set to NULL
    if (new_image ["Eye"] == "Laser" or new_image ["Eye"] == "Laser_B") and new_image ["EyeA"] == "Bandit":
        new_image ["EyeA"] = "Null_EyeA"
        
    if (new_image ["Eye"] == "Laser" or new_image ["Eye"] == "Laser_B") and new_image ["EyeA"] == "Scar":
        new_image ["EyeA"] = "Null_EyeA"
   
    # Laser Eye + Blindfold NULL
    if new_image ["Eye"] == "Laser" and (new_image ["EyeCover"] == "Red_Bf" or new_image ["EyeCover"] == "White_Bf"):
        new_image ["EyeCover"] = "Null_EyeC"
    
    # Alien Conflicts
    elif body_type_choice == "Alien":
        new_image ["FacialH"] = "Null_FacialH"
        new_image ["HairM"] = "Null_HairM"
        new_image ["HairF"] = "Null_HairF"
        new_image ["Nose"] = "Alien_Nose"
        new_image ["Body"] = "White"
        new_image ["Eye"] = "Alien"
        if new_image ["FaceCover"] in Cy_Faces:
            new_image ["Mouth"] = "Neutral"
            new_image ["Nose"] = "Alien_Nose"
            new_image ["Eye"] = "Alien"
            new_image ["FaceCover"] = "Null_FaceC"
        if new_image ["EyeA"] == "Bandit":
            new_image ["EyeA"] = "Null_EyeA"
            
    # Tulip Conflicts 
    elif body_type_choice == "Tulip":
        new_image ["FacialH"] = "Null_FacialH"
        new_image ["HairM"] = "Null_HairM"
        new_image ["HairF"] = "Null_HairF"
        new_image ["Nose"] = "Tulip_Nose"
        new_image ["Body"] = "Dirt_Body"
        new_image ["Eye"] = "Black_Eye"
        new_image ["Hat"] = random.choices(tulip, tulip_weights)[0]
        if new_image ["FaceCover"] in Cy_Faces:
            new_image ["Mouth"] = "Neutral"
            new_image ["Nose"] = "Tulip_Nose"
            new_image ["Eye"] = random.choices(eye, eye_weights)[0]
            new_image ["FaceCover"] = "Null_FaceC"

    if new_image in all_images:
        return create_new_image()
    else:
        return new_image
    
    
# Generate the unique combinations based on trait weightings
for i in range(TOTAL_IMAGES): 
    
    new_trait_image = create_new_image()
    
    all_images.append(new_trait_image)
    


In [ ]:
# Returns true if all images are unique
def all_images_unique(all_images):
    seen = list()
    return not any(i in seen or seen.append(i) for i in all_images)

print("Are all images unique?", all_images_unique(all_images))

In [ ]:
# Add token Id to each image
i = 0
for item in all_images:
    item["tokenId"] = i
    i = i + 1

In [ ]:
# Prints image token and trait for each one

print(all_images)

In [ ]:
# Returns all of the trait counts

background_count = {}
for item in background:
    background_count[item] = 0
    
body_count = {}
for item in body:
    body_count[item] = 0
    
body_type_count = {}
for item in body_type:
    body_type_count[item] = 0

eye_count = {}
for item in eye:
    eye_count[item] = 0
    
eye_c_count = {}
for item in eye_c:
    eye_c_count[item] = 0

eye_a_count = {}
for item in eye_a:
    eye_a_count[item] = 0
    
facial_h_count = {}
for item in facial_h:
    facial_h_count[item] = 0

mouth_count = {}
for item in mouth:
    mouth_count[item] = 0

mouth_a_count = {}
for item in mouth_a:
    mouth_a_count[item] = 0
    
nose_count = {}
for item in nose:
    nose_count[item] = 0
    
face_cover_count = {}
for item in face_cover:
    face_cover_count[item] = 0
    
hair_m_count = {}
for item in hair_m:
    hair_m_count[item] = 0
    
hair_f_count = {}
for item in hair_f:
    hair_f_count[item] = 0
    
hat_count = {}
for item in hat:
    hat_count[item] = 0
    
for image in all_images:
    background_count[image["Background"]] += 1
    body_count[image["Body"]] += 1
    body_type_count[image["BodyType"]] += 1
    eye_count[image["Eye"]] += 1
    eye_c_count[image["EyeCover"]] += 1
    eye_a_count[image["EyeA"]] += 1
    facial_h_count[image["FacialH"]] += 1
    mouth_count[image["Mouth"]] += 1
    mouth_a_count[image["MouthA"]] += 1
    nose_count[image["Nose"]] += 1
    face_cover_count[image["FaceCover"]] += 1
    hair_m_count[image["HairM"]] += 1
    hair_f_count[image["HairF"]] += 1
    hat_count[image["Hat"]] += 1
    
print(background_count)
print(body_count)
print(body_type_count)
print(eye_count)
print(eye_c_count)
print(eye_a_count)
print(facial_h)
print(mouth_count)
print(mouth_a_count)
print(nose_count)
print(face_cover_count)
print(hair_m_count)
print(hair_f_count)
print(hat_count)

In [ ]:
#### Generate Metadata for all Traits 
METADATA_FILE_NAME = './metadata/all-traits.json'; 
with open(METADATA_FILE_NAME, 'w') as outfile:
    json.dump(all_images, outfile, indent=4)

In [ ]:
  
    
#### Generate Images    
for item in all_images:

    im1 = Image.open(f'./trait-layers/Backgrounds/{background_files[item["Background"]]}.jpg').convert('RGBA')
    im2 = Image.open(f'./trait-layers/Body/{body_files[item["Body"]]}.png').convert('RGBA')
    im3 = Image.open(f'./trait-layers/Eye/{eye_files[item["Eye"]]}.png').convert('RGBA')
    im4 = Image.open(f'./trait-layers/FacialH/{facial_h_files[item["FacialH"]]}.png').convert('RGBA')
    im5 = Image.open(f'./trait-layers/Eye_Accessories/{eye_a_files[item["EyeA"]]}.png').convert('RGBA')
    im6 = Image.open(f'./trait-layers/Eye_Cover/{eye_c_files[item["EyeCover"]]}.png').convert('RGBA')
    im7 = Image.open(f'./trait-layers/Face_Cover/{face_cover_files[item["FaceCover"]]}.png').convert('RGBA')
    im8 = Image.open(f'./trait-layers/Nose/{nose_files[item["Nose"]]}.png').convert('RGBA')
    im9 = Image.open(f'./trait-layers/Mouth/{mouth_files[item["Mouth"]]}.png').convert('RGBA')
    im10 = Image.open(f'./trait-layers/HairM/{hair_m_files[item["HairM"]]}.png').convert('RGBA')
    im11 = Image.open(f'./trait-layers/HairF/{hair_f_files[item["HairF"]]}.png').convert('RGBA')
    im12 = Image.open(f'./trait-layers/Hat/{hat_files[item["Hat"]]}.png').convert('RGBA')
    im13 = Image.open(f'./trait-layers/Mouth_Accessories/{mouth_a_files[item["MouthA"]]}.png').convert('RGBA')
    

    # Create each composite
    
    # MOST IMPORTANT PART TO IMAGE PROCCESS
    com1 = Image.alpha_composite(im1, im2) # Background + Body
    com2 = Image.alpha_composite(com1, im3) # + Eye
    com3 = Image.alpha_composite(com2, im4) # + Facial Hair
    com4 = Image.alpha_composite(com3, im5) # + Eye Accessory
    com5 = Image.alpha_composite(com4, im6) # + Eye Cover
    com6 = Image.alpha_composite(com5, im7) # + Face Cover
    com7 = Image.alpha_composite(com6, im8) # + Nose
    com8 = Image.alpha_composite(com7, im9) # + Mouth
    com9 = Image.alpha_composite(com8, im10) # + Male Hair
    com10 = Image.alpha_composite(com9, im11) # + Female Hair
    com11 = Image.alpha_composite(com10, im12) # + Hat
    com12 = Image.alpha_composite(com11, im13) # + Mouth Accessory


    #Convert to RGB
    rgb_im = com12.convert('RGB')
    file_name = str(item["tokenId"]) + ".png"
    rgb_im.save("./images/" + file_name)
    
    
    

In [5]:
#### Generate Metadata for each Image    

f = open('./metadata/all-traits.json',) 
data = json.load(f)


IMAGES_IPFS_HASH = "ENTER IPFS IMG FOLDER HASH HERE"
PROJECT_NAME = "DigitalDoge"

def getAttribute(key, value):
    return {
        "trait_type": key,
        "value": value
    }
for i in data:
    token_id = i['tokenId']
    token = {
        "image" : 'ipfs://' + IMAGES_IPFS_HASH + '/' + str(token_id) + '.png',
        "description": "Discover a collection of 10,000 DigitalDoges each with its own unique traits.",
        "tokenId": token_id,
        "name": PROJECT_NAME + ' #' + str(token_id),
        "attributes": []
    }
    token["attributes"].append(getAttribute("Background", i["Background"]))
    token["attributes"].append(getAttribute("Body", i["Body"]))
    token["attributes"].append(getAttribute("Type", i["BodyType"]))

    # to reduce code create dictionary with all null values to skip
    if i["Eye"] == "Null_Eye": # Need to be more efficient for big O
        pass
    else:
        token["attributes"].append(getAttribute("Eye", i["Eye"]))
    if i["EyeCover"] == "Null_EyeC":
        pass
    else:
        token["attributes"].append(getAttribute("Sunglasses & Stuff", i["EyeCover"]))
    if i["EyeA"] == "Null_EyeA":
        pass
    else:
        token["attributes"].append(getAttribute("Eye Accessories", i["EyeA"]))
    if i["FacialH"] == "Null_FacialH":
        pass
    else:
        token["attributes"].append(getAttribute("Face Accessories", i["FacialH"]))
    if i["Mouth"] == "Null_Mouth":
        pass
    else:
        token["attributes"].append(getAttribute("Mouth", i["Mouth"]))
    if i["MouthA"] == "Null_MouthA":
        pass
    else:
        token["attributes"].append(getAttribute("Mouth Accessories", i["MouthA"]))
    if i["Nose"] == "Null_Nose":
        pass
    else:
        token["attributes"].append(getAttribute("Nose", i["Nose"]))
    if i["FaceCover"] == "Null_FaceC":
        pass
    else:
        token["attributes"].append(getAttribute("Face Cover", i["FaceCover"]))
    if i["HairM"] == "Null_HairM":
        pass
    else:
        token["attributes"].append(getAttribute("Male Hair", i["HairM"]))
    if i["HairF"] == "Null_HairF":
        pass
    else:
        token["attributes"].append(getAttribute("Female Hair", i["HairF"]))
    if i["Hat"] == "Null_Hat":
        pass
    else:
        token["attributes"].append(getAttribute("Hat", i["Hat"]))

    with open('./metadata/' + str(token_id) + '.json', 'w') as outfile:
        json.dump(token, outfile, indent=4)
f.close()